In [1]:
import datajoint as dj
import numpy as np
import time
import pymeshfix

dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config["display.limit"] = 20
    
#schema = dj.schema('microns_ta3p100')
#ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")

Connecting celiib@10.28.0.34:3306


In [2]:
@schema
class PymeshfixDecimatedExcitatoryStitchedMesh(dj.Computed):
    definition = """
    -> pinky.Decimation35ExcitatoryStitchedMesh               
    ---
    n_vertices           : bigint  #total number of vertices after pymeshfix cleaning                     
    n_triangles          : bigint  #total number of faces after pymeshfix cleaning                    
    vertices             : longblob #vertices after pymeshfix cleaning                  
    triangles            : longblob #faces after pymeshfix cleaning  
    perc_vertices_remaining  : decimal(6,5) #number of vertices still present after pymeshfix cleaning
    perc_triangles_remaining  : decimal(6,5) #number of faces still present after pymeshfix cleaning
    
    """
    
    #getting the keys from the following three tables
    #key_source = ta3p100.Mesh & ta3p100.CurrentSegmentation & ta3p100.AllenSomaClass
    
    def make(self, key):
        start_time = time.time()
        
        print("\n\n*****Starting pymeshfix algorithm for " + str(key["segment_id"]) + "******")
        
        joincomp = True
        remove_smallest_components = False
        
        
        #get the vertices 
        mesh = (pinky.Decimation35ExcitatoryStitchedMesh & key).fetch1()
        vertices, triangles = mesh['vertices'], mesh['triangles']
        meshfix = pymeshfix.MeshFix(vertices,triangles)
        verbose=False
        meshfix.repair(verbose,joincomp,remove_smallest_components)
        new_verts = meshfix.v
        new_faces = meshfix.f

        
        self.insert1(dict(key,
                          n_vertices=len(new_verts),
                          n_triangles=len(new_faces),
                          vertices=new_verts,
                          triangles=new_faces,
                         perc_vertices_remaining=len(new_verts)/len(vertices),
                         perc_triangles_remaining=len(new_faces)/len(triangles)),skip_duplicates=True)
        
        
        print(f"Finished pymeshfix algorithm: {time.time() - start_time}")
        


In [3]:
start = time.time()
PymeshfixDecimatedExcitatoryStitchedMesh.populate(reserve_jobs=True)
print(time.time() - start)



*****Starting pymeshfix algorithm for 648518346349471500******
Finished pymeshfix algorithm: 48.69105052947998


*****Starting pymeshfix algorithm for 648518346349473813******
Finished pymeshfix algorithm: 89.96298265457153


*****Starting pymeshfix algorithm for 648518346349479056******
Finished pymeshfix algorithm: 114.3127670288086


*****Starting pymeshfix algorithm for 648518346349486885******
Finished pymeshfix algorithm: 216.49163365364075


*****Starting pymeshfix algorithm for 648518346349498239******
Finished pymeshfix algorithm: 314.2478618621826


*****Starting pymeshfix algorithm for 648518346349502434******
Finished pymeshfix algorithm: 150.27255392074585


*****Starting pymeshfix algorithm for 648518346349505250******
Finished pymeshfix algorithm: 593.0584871768951
1534.404931306839


In [4]:
#(schema.jobs & "table_name='__pymeshfix_decimated_excitatory_stitched_mesh'").delete()